# Object Pool

In [2]:
from queue import Queue

class Proxy:
    """Wraps original object with context manager that return the object to the
    pool."""
    def __init__(self, obj, pool):
        self._obj = obj
        self._pool = pool

    def __enter__(self):
        return self._obj

    def __exit__(self, typ, val, tb):
        self._pool._put(self._obj)


# useful in managing resources, for example limiting or limited connections to something        
class Pool:
    """Pool of objects"""
    def __init__(self, objects):
        self._queue = Queue()
        for obj in objects:
            self._queue.put(obj)

    def lease(self):
        """Lease an object from the pool, should be used as connection manager. e.g.:
            with pool.lease() as conn:
                cur = conn.cursor()
                cur.execute('SELECT ...')
        """
        return Proxy(self._queue.get(), self)

    def _put(self, obj):
        self._queue.put(obj)

# http://pythonwise.blogspot.com/2016/09/simple-object-pools.html
# Miki Tebeka

In [4]:
from threading import Thread, Barrier
from time import sleep
from random import random

n = 9
b = Barrier(n)
p = Pool([1, 2, 3])

def worker(n, barrier, pool):
    barrier.wait()  # Wait for all threads to be ready
    sleep(random() / 10)
    with pool.lease() as val:
        print(f"worker {n:d} got resource {val:d} ")

for i in range(n):
    Thread(target=worker, args=(i, b, p)).start()

worker 8 got resource 1 
worker 7 got resource 2 
worker 4 got resource 3 
worker 6 got resource 1 
worker 2 got resource 2 
worker 0 got resource 3 
worker 3 got resource 1 
worker 5 got resource 2 
worker 1 got resource 3 
